# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [3]:
# Write your code below.

import os
# Set working dir the same as  `.env` 
os.chdir('../../01_materials/labs/')
    
%load_ext dotenv 
%dotenv .env


In [4]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob

# Write your code below.
# Load the PRICE_DATA
PRICE_DATA = os.getenv("PRICE_DATA")

# Get all parquet files  using Glob
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
parquet_files

['../../05_src/data/prices/AMZN/AMZN_2023.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2015.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2014.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2022.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2016.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2020.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2021.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2017.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2024.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2013.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2018.parquet',
 '../../05_src/data/prices/AMZN/AMZN_2019.parquet',
 '../../05_src/data/prices/RCL/RCL_2019.parquet',
 '../../05_src/data/prices/RCL/RCL_2018.parquet',
 '../../05_src/data/prices/RCL/RCL_2013.parquet',
 '../../05_src/data/prices/RCL/RCL_2024.parquet',
 '../../05_src/data/prices/RCL/RCL_2021.parquet',
 '../../05_src/data/prices/RCL/RCL_2017.parquet',
 '../../05_src/data/prices/RCL/RCL_2016.parquet',
 '../../05_src/data/prices

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Since the task requires 'Close' and 'Adj Close', 
# adding the unadjusted 'Close' column as 'Close' into the adjusted data from the data_engineering file.
import os
import pandas as pd
import yfinance as yf

# 1. List to hold final results
adjusted_px_list = list()
unadjusted_close_list = list()

# 2. Load tickers from the CSV file
ticker_file = os.getenv("TICKERS")
tickers = pd.read_csv(ticker_file).sample(50, random_state=42)

for k, row in tickers.iterrows():  # Produces an iterator that returns index and row
    ticker = row['ticker']
    print(f'Processing {ticker}')
    
    # 3. Download adjusted data
    stock = yf.Ticker(ticker)
    adjusted_px = (stock.history(start="2013-12-01", end="2024-02-01")
                   .reset_index()   # Reset index to get date as a column
                   .assign(ticker=ticker))    # Add ticker
    
    # Rename columns for clarity
    adjusted_px.rename(columns={'Close': 'Adj_Close'}, inplace=True)
    
    # 4. Download unadjusted data
    unadjusted_px = (stock.history(start="2013-12-01", end="2024-02-01", auto_adjust=False)
                     .reset_index()   # Reset index to get date as a column
                     .assign(ticker=ticker))    # Add ticker
    
    # 5. Merge adjusted data with unadjusted 'Close' column
    combined_px = adjusted_px.merge(unadjusted_px[['Date', 'Close']], on='Date', suffixes=('', '_unadjusted'))
    
    if combined_px.shape[0] == 0:
        print(f'No data for {ticker}')  # Validate: do not fail silently.
        continue
    
    # print(f'Downloaded {combined_px.shape}.')
    adjusted_px_list.append(combined_px)

# 6. Concatenate all dataframes
final_px_dt = pd.concat(adjusted_px_list, axis=0)
# print(f'Final shape {final_px_dt.shape}.')


In [29]:
# 7. Convert to Dask DataFrame and store as Parquet
px_dd = dd.from_pandas(final_px_dt, npartitions = len(tickers))
temp = os.getenv("TEMP_DATA")
os.makedirs(temp, exist_ok=True)
stock_path = os.path.join(temp, "stock_px.csv")
parquet_path = os.path.join(temp, "stock_px.parquet")
px_dd.to_parquet(parquet_path, engine = "pyarrow")

# 8. CLean up before start
PRICE_DATA = os.getenv("PRICE_DATA")
import shutil
if os.path.exists(PRICE_DATA):
    shutil.rmtree(PRICE_DATA)


In [30]:

# 9. Process and store data by ticker and year
for ticker in px_dd.ticker.unique():
    ticker_dt = px_dd[px_dd.ticker == ticker]
    ticker_dt = ticker_dt.assign(year = ticker_dt.Date.dt.year)
    for yr in ticker_dt.year.unique():
        yr_dt = ticker_dt[ticker_dt.year == yr]
        yr_path = os.path.join(PRICE_DATA, ticker, f"{ticker}_{yr}.parquet")
        os.makedirs(os.path.dirname(yr_path), exist_ok=True)
        yr_dt.to_parquet(yr_path, engine = "pyarrow")

# 10. Get all parquet files using Glob
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
parquet_files

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8e8455aa30>>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
import dask.dataframe as dd

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [24]:
# Write your code below.

# 1. Load the Parquet files into a Dask DataFrame and set index to 'ticker'
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px.compute()

# 2. Add lagged features, calculate returns, and high-low range
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag = x['Close'].shift(1),
        Adj_Close_lag = x['Adj_Close'].shift(1)
        )
).assign(
    returns = lambda x: x['Adj_Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range=lambda x: x['High'] - x['Low'],
))

dd_feat


ValueError: empty urlpath sequence

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

# Convert to Pandas DataFrame
pd_feat = dd_feat.compute()

# 3. Add rolling average return calculation with a window of 10 days
pd_feat['rolling_avg_return'] = pd_feat.groupby('ticker')['returns'] \
                                .rolling(10) \
                                .mean() \
                                .reset_index(level=0, drop=True)

# Display the resulting DataFrame
pd_feat.head()


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.